In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


## GJENSTÅR

python3 utils/run_abalone.py --model beta_horseshoe_tanh_nodewise --output_dir results/regression/single_layer/tanh/abalone 

python3 utils/run_abalone.py --model beta_student_t_tanh_nodewise --output_dir results/regression/single_layer/tanh/abalone 

python3 utils/run_abalone.py --model dirichlet_student_t_tanh_nodewise --output_dir results/regression/single_layer/tanh/abalone 

In [ ]:
data_dir = f"datasets/abalone"
results_dir_relu = "results/regression/single_layer/relu/abalone"
results_dir_tanh = "results/regression/single_layer/tanh/abalone"

# model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T", "Beta Horseshoe", "Beta Student T"]
# model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
#model_names_relu = ["Dirichlet Horseshoe", "Dirichlet Student T", "Beta Horseshoe", "Beta Student T"]
model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]


full_config_path = "abalone_N334_p8"
# relu_fit = get_model_fits(
#     config=full_config_path,
#     results_dir=results_dir_relu,
#     models=model_names_relu,
#     include_prior=False,
# )

tanh_fit = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh,
    include_prior=False,
)

In [ ]:
data_dir = f"datasets/abalone"
results_dir_relu = "results/regression/single_layer/relu/abalone"
results_dir_tanh = "results/regression/single_layer/tanh/abalone"

# model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T", "Beta Horseshoe", "Beta Student T"]
# model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
# model_names_relu_nodewise = ["Dirichlet Horseshoe", "Dirichlet Student T", "Beta Horseshoe", "Beta Student T"]
model_names_tanh_nodewise = ["Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise", "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"] #"Dirichlet Student T tanh nodewise", 


full_config_path = "abalone_N334_p8"
# relu_fit = get_model_fits(
#     config=full_config_path,
#     results_dir=results_dir_relu,
#     models=model_names_relu,
#     include_prior=False,
# )

tanh_fit_nodewise = get_model_fits(
    config=full_config_path,
    results_dir=results_dir_tanh,
    models=model_names_tanh_nodewise,
    include_prior=False,
)


## TEST NEW PRUNING SCHEME 

In [4]:
import numpy as np

def build_global_mask_from_posterior(
    W_samples,
    sparsity,
    method="Eabs",          # "Eabs" or "Eabs_stability"
    stability_quantile=0.1, # used if method="Eabs_stability"
    prune_smallest=True
):
    """
    W_samples: array (S, ..., ...) posterior draws of a weight matrix.
    sparsity: fraction to prune (q). Keeps (1-q).
    Returns mask with same trailing shape as one draw, dtype float {0,1}.
    """
    assert 0.0 <= sparsity < 1.0
    S = W_samples.shape[0]
    W_abs = np.abs(W_samples)  # (S, ...)

    # Importance score a = E|w|
    a = W_abs.mean(axis=0)     # (..., ...)

    if method == "Eabs":
        score = a
    elif method == "Eabs_stability":
        # Stability proxy pi = P(|w| > t), where t is a small global quantile of |w|
        t = np.quantile(W_abs.reshape(S, -1), stability_quantile)
        pi = (W_abs > t).mean(axis=0)
        # Combine: emphasize both "large on average" and "consistently non-tiny"
        score = a * pi
    else:
        raise ValueError("method must be 'Eabs' or 'Eabs_stability'")

    # Decide how many to prune
    num_params = score.size
    k_prune = int(np.floor(sparsity * num_params))
    if k_prune == 0:
        return np.ones_like(score, dtype=float)

    flat = score.reshape(-1)

    if prune_smallest:
        # prune lowest scores
        thresh = np.partition(flat, k_prune - 1)[k_prune - 1]
        mask = (score > thresh).astype(float)
        # if ties create too many kept/pruned, fix deterministically
        # (rare but possible with many equal scores)
        if mask.sum() > num_params - k_prune:
            # drop some tied-at-threshold entries
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_drop = int(mask.sum() - (num_params - k_prune))
            if need_drop > 0:
                mask_flat = mask.reshape(-1)
                mask_flat[idx_tied[:need_drop]] = 0.0
                mask = mask_flat.reshape(score.shape)
        elif mask.sum() < num_params - k_prune:
            # add some tied entries if we kept too few
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_add = int((num_params - k_prune) - mask.sum())
            if need_add > 0:
                mask_flat = mask.reshape(-1)
                # add back from tied
                add_candidates = idx_tied[mask_flat[idx_tied] == 0.0]
                mask_flat[add_candidates[:need_add]] = 1.0
                mask = mask_flat.reshape(score.shape)
    else:
        # prune largest (not typical)
        thresh = np.partition(flat, num_params - k_prune)[num_params - k_prune]
        mask = (score < thresh).astype(float)

    return mask


def precompute_global_masks(
    all_fits,
    model,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    """
    Returns dict: sparsity -> (mask_W1, mask_W2 or None)
    """
    fit = all_fits[model]["posterior"]

    W1_samples = fit.stan_variable("W_1")  # (S, P, H)
    W2_samples = fit.stan_variable("W_L")  # (S, H, O) or (S, H) depending on O

    masks = {}
    for q in sparsity_levels:
        mask_W1 = build_global_mask_from_posterior(W1_samples, q, method=method)
        mask_W2 = None
        if prune_W2:
            mask_W2 = build_global_mask_from_posterior(W2_samples, q, method=method)
        masks[q] = (mask_W1, mask_W2)
    return masks


In [5]:
from utils.generate_data import load_abalone_regression_data
def _logsumexp(a, axis=None):
    amax = np.max(a, axis=axis, keepdims=True)
    out = amax + np.log(np.sum(np.exp(a - amax), axis=axis, keepdims=True))
    return np.squeeze(out, axis=axis)

def gaussian_nll_pointwise(y, mu, sigma):
    return 0.5*np.log(2*np.pi*(sigma**2)) + 0.5*((y-mu)**2)/(sigma**2)

def compute_sparse_metrics_results_globalmask_large_eval(
    models, all_fits, forward_pass,
    sparsity=0.0,
    masks_cache=None,
    prune_W2=False,
    compute_nll=True,
    noise_var_name="sigma",
    frac = 1.0
):
    """
    Evaluate on a large generated test set instead of the stored tiny X_test/y_test.
    Assumes model was trained on standardized y if standardize_y=True.
    """
    posterior_means = []
    # Build large eval set consistent with training split standardization
    X_train, X_test, y_train, y_test = load_abalone_regression_data(standardized=False, frac=frac)
    #y_std = y_train.std()
    #y_mean = y_train.mean()

    for model in models:
        try:
            fit = all_fits[model]['posterior']
            W1_samples = fit.stan_variable("W_1")           # (S, P, H)
            W2_samples = fit.stan_variable("W_L")           # (S, H, O)
            b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
            b2_samples = fit.stan_variable("output_bias")   # (S, O)

            noise_samples = None
            if compute_nll:
                try:
                    noise_samples = fit.stan_variable(noise_var_name).squeeze()
                except Exception:
                    noise_samples = None
        except KeyError:
            print(f"[SKIP] Model or posterior not found: -> {model}")
            continue

        S = W1_samples.shape[0]
        y_hats = np.zeros((S, y_test.shape[0]))

        mask_W1 = mask_W2 = None
        if masks_cache is not None and sparsity > 0.0:
            mask_W1, mask_W2 = masks_cache[(model)][sparsity]

        for i in range(S):
            W1 = W1_samples[i]
            W2 = W2_samples[i]

            if mask_W1 is not None:
                W1 = W1 * mask_W1
            if prune_W2 and (mask_W2 is not None):
                W2 = W2 * mask_W2

            y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i]).squeeze()
            y_hats[i] = y_hat

        # posterior mean RMSE (standardized scale)
        posterior_mean = y_hats.mean(axis=0)
        posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test)**2))

        out_pm = {
            'N': X_train.shape[0],
            'model': model,
            'sparsity': sparsity,
            'n_eval': y_test.shape[0],
            'posterior_mean_rmse': posterior_mean_rmse,
        }

        if compute_nll:
            if noise_samples is None:
                sig_s = np.ones(S)
            else:
                sig_s = np.asarray(noise_samples).reshape(-1)[:S]

            # Expected NLL
            nll_draws = np.array([
                gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i]).mean()
                for i in range(S)
            ])
            expected_nll = nll_draws.mean()

            # Predictive (mixture) NLL
            loglik = -np.stack([
                gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i])
                for i in range(S)
            ], axis=0)  # (S, n_eval)
            lppd = (_logsumexp(loglik, axis=0) - np.log(S)).mean()
            predictive_nll = -lppd

            out_pm["expected_nll"] = expected_nll
            out_pm["predictive_nll"] = predictive_nll


        posterior_means.append(out_pm)

    return pd.DataFrame(posterior_means)


In [6]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

def build_masks_cache_for_all(
    all_fits,
    models,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    masks_cache = {}
    for model in models:
        try:
            masks_cache[(model)] = precompute_global_masks(
                all_fits=all_fits,
                model=model,
                sparsity_levels=sparsity_levels,
                prune_W2=prune_W2,
                method=method
            )
        except KeyError:
            print(f"[SKIP MASKS] Missing fit for -> {model}")
    return masks_cache

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# Precompute masks once
# masks_relu = build_masks_cache_for_all(relu_fit, model_names_relu, sparsity_levels, prune_W2=False)

masks_tanh = build_masks_cache_for_all(tanh_fit, model_names_tanh, sparsity_levels, prune_W2=False)
masks_tanh_nodewise = build_masks_cache_for_all(tanh_fit_nodewise, model_names_tanh_nodewise, sparsity_levels, prune_W2=False)


In [12]:
# df_post_relu = {}

# for q in sparsity_levels:
#     df_post_relu[q] = compute_sparse_metrics_results_globalmask_large_eval(
#         models=model_names_relu,
#         all_fits=relu_fit,
#         forward_pass=forward_pass_relu,
#         sparsity=q,
#         masks_cache=masks_relu,
#         prune_W2=False,
#         compute_nll=True,
#         noise_var_name="sigma",
#         frac=0.1
#     )

df_post_tanh = {}

for q in sparsity_levels:
    df_post_tanh[q] = compute_sparse_metrics_results_globalmask_large_eval(
        models=model_names_tanh,
        all_fits=tanh_fit,
        forward_pass=forward_pass_tanh,
        sparsity=q,
        masks_cache=masks_tanh,
        prune_W2=False,
        compute_nll=True,
        noise_var_name="sigma",
        frac=0.1
    )

In [13]:
df_post_tanh_nodewise = {}

for q in sparsity_levels:
    df_post_tanh_nodewise[q] = compute_sparse_metrics_results_globalmask_large_eval(
        models=model_names_tanh_nodewise,
        all_fits=tanh_fit_nodewise,
        forward_pass=forward_pass_tanh,
        sparsity=q,
        masks_cache=masks_tanh_nodewise,
        prune_W2=False,
        compute_nll=True,
        noise_var_name="sigma",
        frac=0.1
    )

In [14]:
# df_post_relu_full = pd.concat(
#     [df.assign(sparsity=sparsity) for sparsity, df in df_post_relu.items()],
#     ignore_index=True
# )

df_post_tanh_full = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_post_tanh.items()],
    ignore_index=True
)

df_post_tanh_full_nodewise = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_post_tanh_nodewise.items()],
    ignore_index=True
)

In [15]:
df_all = pd.concat([df_post_tanh_full,df_post_tanh_full_nodewise])

In [ ]:
df_post_tanh_full[df_post_tanh_full['sparsity']==0.0].sort_values(by="posterior_mean_rmse").round(3)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",  
    "Beta Student T tanh nodewise": "C5",                 
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

rmse_col = "posterior_mean_rmse"
#pll_col  = "predictive_nll"   # use *_orig if you prefer

df = df_all.copy()
#df["pll"] = -df[pll_col]  # convert NLL → PLL

df = df[df["sparsity"].isin(sparsity_levels)]
df["sparsity"] = pd.Categorical(df["sparsity"],
                                categories=sparsity_levels,
                                ordered=True)


fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True)

for model, g in df.groupby("model", sort=False):
    if model in ["Gaussian tanh", "Dirichlet Student T tanh nodewise", "Dirichlet Student T tanh", "Beta Student T tanh", "Beta Student T tanh nodewise"]:
       continue
    g = g.sort_values("sparsity")
    if "nodewise" in model:
        axes[0].plot(g["sparsity"], g[rmse_col], marker="v", label=model, linestyle='dashed', color = colors[model])
        axes[1].plot(g["sparsity"], g["predictive_nll"],    marker="v", label=model, linestyle='dashed', color = colors[model])
    else:
        axes[0].plot(g["sparsity"], g[rmse_col], marker="o", label=model, color = colors[model])
        axes[1].plot(g["sparsity"], g["predictive_nll"],    marker="o", label=model, color = colors[model])
    
axes[0].set_title(f"RMSE vs sparsity")
axes[1].set_title(f"PNLL vs sparsity")

axes[0].set_ylabel("RMSE")
axes[1].set_ylabel("PNLL")

for ax in axes:
    ax.set_xlabel("sparsity")
    ax.set_xticks(sparsity_levels)
    ax.grid(True, linestyle="--", linewidth=0.5)

axes[0].legend(loc="upper left",
                #bbox_to_anchor=(1.02, 0.5),
                frameon=False)
axes[1].legend(loc="upper left",
                #bbox_to_anchor=(1.02, 0.5),
                frameon=False)

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",  
    "Beta Student T tanh nodewise": "C5",                 
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

rmse_col = "posterior_mean_rmse"
#pll_col  = "predictive_nll"   # use *_orig if you prefer

df = df_all.copy()
#df["pll"] = -df[pll_col]  # convert NLL → PLL

df = df[df["sparsity"].isin(sparsity_levels)]
df["sparsity"] = pd.Categorical(df["sparsity"],
                                categories=sparsity_levels,
                                ordered=True)


fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True)

for model, g in df.groupby("model", sort=False):
    if model in ["Gaussian tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Horseshoe tanh", "Beta Horseshoe tanh", "Beta Horseshoe tanh nodewise"]:
       continue
    g = g.sort_values("sparsity")
    if "nodewise" in model:
        axes[0].plot(g["sparsity"], g[rmse_col], marker="v", label=model, linestyle='dashed', color = colors[model])
        axes[1].plot(g["sparsity"], g["predictive_nll"],    marker="v", label=model, linestyle='dashed', color = colors[model])
    else:
        axes[0].plot(g["sparsity"], g[rmse_col], marker="o", label=model, color = colors[model])
        axes[1].plot(g["sparsity"], g["predictive_nll"],    marker="o", label=model, color = colors[model])
    
axes[0].set_title(f"RMSE vs sparsity")
axes[1].set_title(f"PNLL vs sparsity")

axes[0].set_ylabel("RMSE")
axes[1].set_ylabel("PNLL")

for ax in axes:
    ax.set_xlabel("sparsity")
    ax.set_xticks(sparsity_levels)
    ax.grid(True, linestyle="--", linewidth=0.5)

axes[0].legend(loc="upper left",
                #bbox_to_anchor=(1.02, 0.5),
                frameon=False)
axes[1].legend(loc="upper left",
                #bbox_to_anchor=(1.02, 0.5),
                frameon=False)

plt.tight_layout()
plt.show()


## VISUALIZE THE PRUNING

In [26]:
feature_names = ["Sex", "Length", "Diameter", "Height",
        "Whole weight", "Shucked weight", "Viscera weight",
        "Shell weight"] 

In [ ]:
def _node_sizes_from_degree(deg, base=80, scale=30):
    # deg: array of degrees (>=0)
    return base + scale * deg

def plot_pruned_network_panels(
    masks_for_model,                 # dict: q -> (mask_W1, mask_W2)
    q_levels=(0.0, 0.5, 0.8, 0.9),    # 4 values recommended
    feature_names=None,              # list length P
    hidden_names=None,               # list length H (optional)
    hidden_bias_meanabs=None,         # array length H (optional): E|b1_j| (or similar)
    title_prefix="",
):
    q_levels = list(q_levels)

    # infer dimensions
    mask_W1_0, mask_W2_0 = masks_for_model[q_levels[0]]
    P, H = mask_W1_0.shape
    O = 1  # scalar output

    if feature_names is None:
        feature_names = [f"x{i+1}" for i in range(P)]
    if hidden_names is None:
        hidden_names = [f"h{j+1}" for j in range(H)]

    # fixed node positions (same in every panel)
    # y arranged top-to-bottom
    y_inputs = np.linspace(1.0, 0.0, P)
    y_hidden = np.linspace(1.0, 0.0, H)
    x_in, x_hid, x_out = 0.0, 1.0, 2.0

    pos_in = np.column_stack([np.full(P, x_in), y_inputs])
    pos_h  = np.column_stack([np.full(H, x_hid), y_hidden])
    pos_o  = np.array([[x_out, 0.5]])

    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    axes = axes.ravel()

    for ax, q in zip(axes, q_levels):
        mask_W1, mask_W2 = masks_for_model[q]
        if mask_W2 is None:
            # if you didn't prune W2, treat as all ones (H x 1)
            mask_W2 = np.ones((H, 1), dtype=float)

        # degrees
        deg_in = mask_W1.sum(axis=1)             # (P,)
        deg_h_in = mask_W1.sum(axis=0)           # (H,)
        deg_h_out = mask_W2[:, 0].astype(float)  # (H,) since scalar output, 0/1 if pruned

        # node sizes
        s_in = _node_sizes_from_degree(deg_in, base=80, scale=25)
        # Hidden: combine structure + (optional) bias magnitude
        if hidden_bias_meanabs is not None:
            # normalize bias for sizing (robust-ish)
            b = np.asarray(hidden_bias_meanabs).reshape(-1)
            b = b / (np.max(b) + 1e-12)
            s_h = 120 + 180 * b + 20 * deg_h_in
        else:
            s_h = _node_sizes_from_degree(deg_h_in, base=120, scale=20)

        s_o = 180

        # -------- edges: build line segments --------
        segs = []
        # input -> hidden
        ii, jj = np.where(mask_W1 > 0.5)
        for i, j in zip(ii, jj):
            segs.append([pos_in[i], pos_h[j]])

        # hidden -> output (only if mask_W2[j]=1)
        alive_h = np.where(deg_h_out > 0.5)[0]
        for j in alive_h:
            segs.append([pos_h[j], pos_o[0]])

        if segs:
            lc = LineCollection(segs, linewidths=1.0)
            ax.add_collection(lc)

        # -------- nodes --------
        ax.scatter(pos_in[:, 0], pos_in[:, 1], s=s_in)
        ax.scatter(pos_h[:, 0], pos_h[:, 1], s=s_h)
        ax.scatter(pos_o[:, 0], pos_o[:, 1], s=s_o)

        # labels (keep minimal; can comment out if too busy)
        for i, name in enumerate(feature_names):
            ax.text(pos_in[i, 0] - 0.03, pos_in[i, 1], name, ha="right", va="center", fontsize=9)
        # hidden labels only for “alive-ish” units to reduce clutter:
        # (either has incoming or outgoing)
        show_h = np.where((deg_h_in > 0) | (deg_h_out > 0.5))[0]
        for j in show_h:
            ax.text(pos_h[j, 0] + 0.03, pos_h[j, 1], hidden_names[j], ha="left", va="center", fontsize=8)

        ax.text(pos_o[0, 0] + 0.03, pos_o[0, 1], "y", ha="left", va="center", fontsize=10)

        # cosmetics
        ax.set_title(f"{title_prefix}q={q}")
        ax.set_xlim(-0.4, 2.4)
        ax.set_ylim(-0.05, 1.05)
        ax.axis("off")

        # annotate with a quick summary
        kept_W1 = int(mask_W1.sum())
        total_W1 = mask_W1.size
        kept_W2 = int(mask_W2.sum())
        total_W2 = mask_W2.size
        ax.text(
            0.0, -0.12,
            f"W1 kept: {kept_W1}/{total_W1}  |  W2 kept: {kept_W2}/{total_W2}  |  alive hidden (out): {kept_W2}",
            transform=ax.transAxes,
            ha="left", va="top", fontsize=9
        )

    # if fewer than 4 q-levels, hide extra axes
    for k in range(len(q_levels), 4):
        axes[k].axis("off")

    plt.tight_layout()
    plt.show()

model = "Regularized Horseshoe tanh"
masks_for_model = masks_tanh[model]  # dict q -> (mask_W1, mask_W2)
fit = tanh_fit[model]["posterior"]
b1_samples = fit.stan_variable("hidden_bias")   # shape (S, H) or (S, 1, H) depending on your Stan
hidden_bias_meanabs = np.mean(np.abs(b1_samples.reshape(b1_samples.shape[0], -1)), axis=0)


plot_pruned_network_panels(
    masks_for_model=masks_for_model,
    q_levels=[0.8, 0.9],
    feature_names=feature_names,
    hidden_bias_meanabs=hidden_bias_meanabs,  # or None
    title_prefix=f"{model} "
)

In [ ]:
model = "Dirichlet Horseshoe tanh"
masks_for_model = masks_tanh[model]  # dict q -> (mask_W1, mask_W2)
fit = tanh_fit[model]["posterior"]
b1_samples = fit.stan_variable("hidden_bias")   # shape (S, H) or (S, 1, H) depending on your Stan
hidden_bias_meanabs = np.mean(np.abs(b1_samples.reshape(b1_samples.shape[0], -1)), axis=0)

plot_pruned_network_panels(
    masks_for_model=masks_for_model,
    q_levels=[0.8, 0.9],
    feature_names=feature_names,
    hidden_bias_meanabs=hidden_bias_meanabs,  # or None
    title_prefix=f"{model} "
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

def plot_input_survival_curves(
    masks_for_model,                 # dict: q -> (mask_W1, mask_W2)
    feature_names=None,              # list length P
    sparsity_levels=None,            # list of q's (sorted)
    title=None,
):
    if sparsity_levels is None:
        sparsity_levels = sorted(masks_for_model.keys())
    else:
        sparsity_levels = list(sparsity_levels)

    # infer P, H from first mask
    m0 = masks_for_model[sparsity_levels[0]][0]
    P, H = m0.shape

    if feature_names is None:
        feature_names = [f"x{i+1}" for i in range(P)]

    # degrees d_i(q) = sum_j mask[i,j]
    D = np.zeros((P, len(sparsity_levels)), dtype=float)
    for t, q in enumerate(sparsity_levels):
        mask_W1, _ = masks_for_model[q]
        D[:, t] = mask_W1.sum(axis=1)

    # Plot
    fig, ax = plt.subplots(figsize=(9, 5))
    for i in range(P):
        ax.plot(sparsity_levels, D[i, :], marker="o", linewidth=1, label=feature_names[i])

    # optional mean curve
    ax.plot(sparsity_levels, D.mean(axis=0), marker="o", linewidth=2, label="mean (inputs)")

    ax.set_xlabel("sparsity q")
    ax.set_ylabel("surviving connections per input (degree)")
    ax.set_xticks(sparsity_levels)
    ax.set_ylim(-0.5, H + 0.5)
    ax.grid(True, linestyle="--", linewidth=0.5)
    if title:
        ax.set_title(title)

    ax.legend(loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
    plt.tight_layout()
    plt.show()

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# masks_tanh = build_masks_cache_for_all(tanh_fit, model_names_tanh, sparsity_levels, prune_W2=True)
# masks_tanh_nodewise = build_masks_cache_for_all(tanh_fit_nodewise, model_names_tanh_nodewise, sparsity_levels, prune_W2=True)
model = "Regularized Horseshoe tanh"
masks_for_model = masks_tanh[model]  # dict q -> (mask_W1, mask_W2)

# If you want bias sizes: compute once from the fit (example names; adjust to your Stan variables)
fit = tanh_fit[model]["posterior"]
b1_samples = fit.stan_variable("hidden_bias")   # shape (S, H) or (S, 1, H) depending on your Stan
hidden_bias_meanabs = np.mean(np.abs(b1_samples.reshape(b1_samples.shape[0], -1)), axis=0)


#sparsity_levels = [0.0, 0.3, 0.5, 0.7]

plot_input_survival_curves(
    masks_for_model=masks_for_model,
    feature_names=feature_names,
    sparsity_levels=sparsity_levels,
    title=f"{model}: input survival curves"
)


In [ ]:
model = "Dirichlet Horseshoe tanh"
masks_for_model = masks_tanh[model]  # dict q -> (mask_W1, mask_W2)

# If you want bias sizes: compute once from the fit (example names; adjust to your Stan variables)
fit = tanh_fit[model]["posterior"]
b1_samples = fit.stan_variable("hidden_bias")   # shape (S, H) or (S, 1, H) depending on your Stan
hidden_bias_meanabs = np.mean(np.abs(b1_samples.reshape(b1_samples.shape[0], -1)), axis=0)

plot_input_survival_curves(
    masks_for_model=masks_for_model,
    feature_names=feature_names,
    sparsity_levels=sparsity_levels,
    title=f"{model}: input survival curves"
)